In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
import seaborn as sns
plt.rc('font',family='Malgun Gothic')
plt.rc('axes',unicode_minus=False)
import sys, warnings
warnings.filterwarnings("ignore")
import re
import os
import datetime
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_predict, RandomizedSearchCV, GridSearchCV, cross_val_score
from xgboost import XGBClassifier, plot_importance
from lightgbm import LGBMClassifier # 한글 지원 X
from vecstack import stacking, StackingTransformer
from sklearn.metrics import accuracy_score, roc_auc_score
from mlxtend.plotting import plot_decision_regions
from IPython.display import display
import itertools
pd.set_option('display.max_columns', 1200)
np.set_printoptions(threshold = sys.maxsize)
from scipy.stats.mstats import gmean
from scipy.stats import randint as sp_randint
from scipy.stats import uniform
import random
from sklearn.dummy import DummyClassifier
from sklearn import metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA

In [2]:
feature = pd.read_csv('features.csv',encoding='cp949')

In [3]:
feature = feature.fillna(0)

In [4]:
feature

,cust_id,총구매액,구매건수,평균구매액,최대구매액,구매상품종류1,구매상품종류2,구매상품종류3,내점일수,구매주기,주말방문비율,봄-구매비율,여름-구매비율,가을-구매비율,겨울-구매비율,환불금액,환불건수,내점당구매액,내점당구매건수,최근3개월_구매금액,최근3개월_구매건수,최근6개월_구매금액,최근6개월_구매건수,최근12개월_구매금액,최근12개월_구매건수,구매추세,봄_평균구매금액,여름_평균구매금액,가을_평균구매금액,겨울_평균구매금액,가격선호도,시즌선호도,휴면여부,기타_구매금액,기타_구매건수,기타_구매여부,농산물_구매금액,농산물_구매건수,농산물_구매여부,가공식품_구매금액,가공식품_구매건수,가공식품_구매여부,축산가공_구매금액,축산가공_구매건수,축산가공_구매여부,수산품_구매금액,수산품_구매건수,수산품_구매여부,화장품_구매금액,화장품_구매건수,화장품_구매여부,디자이너_구매금액,디자이너_구매건수,디자이너_구매여부,시티웨어_구매금액,시티웨어_구매건수,시티웨어_구매여부,일용잡화_구매금액,일용잡화_구매건수,일용잡화_구매여부,육류_구매금액,육류_구매건수,육류_구매여부,주구매상품,주구매지점,구매순서1위,여부3,구매건수_많은달,구매건수_많은계절,금_평균구매금액,목_평균구매금액,수_평균구매금액,월_평균구매금액,일_평균구매금액,토_평균구매금액,화_평균구매금액,금_구매건수,목_구매건수,수_구매건수,월_구매건수,일_구매건수,토_구매건수,화_구매건수,봄_주요구매상품,여름_주요구매상품,가을_주요구매상품,겨울_주요구매상품,다른지점구매비율,월초_총구매액,월초_구매건수,월초_평균구매액,월중_총구매액,월중_구매건수,월중_평균구매액,월말_총구매액,월말_구매건수,월말_평균구매액
0,0,68282840,74,922741,11264000,35,28,17,19,17,0.527027,0.094595,0.391892,0.378378,0.135135,6860000.0,3.0,3.593834e+06,3.894737,29084140.0,27.0,42601240.0,61.0,68282840,74,0.098039,2.797000e+06,5.832745e+05,7.725586e+05,1.015724e+06,3,2,0,507200.0,13.0,1.0,74360.0,3.0,1.0,35520.0,1.0,1.0,119000.0,2.0,1.0,866000.0,4.0,1.0,22450000.0,7.0,1.0,11264000.0,4.0,1.0,9806000.0,7.0,1.0,0.0,0.0,0.0,567860.0,10.0,1.0,기타,강남점,건강식품,0,10,2,1.890756e+06,610200.0,-3.647033e+05,412440.0,2.338358e+06,6.281607e+05,2.243000e+06,9,6,12,6,12,27,2,화장품,육류,기타,육류,0.040541,24933940.0,29.0,8.597910e+05,29927360.0,32.0,9.352300e+05,13421540.0,13.0,1.032426e+06
1,1,2136000,3,712000,2136000,2,2,2,2,1,0.000000,0.000000,0.000000,1.000000,0.000000,300000.0,1.0,1.068000e+06,1.500000,2136000.0,3.0,2136000.0,3.0,2136000,3,0.250000,0.000000e+00,0.000000e+00,7.120000e+05,0.000000e+00,3,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,스포츠,잠실점,스포츠,1,10,3,0.000000e+00,918000.0,0.000000e+00,300000.0,0.000000e+00,0.000000e+00,0.000000e+00,0,2,0,1,0,0,0,0,0,스포츠,0,0.000000,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,2136000.0,3.0,7.120000e+05
2,2,3197000,4,799250,1639000,1,2,2,2,1,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.0,1.598500e+06,2.000000,0.0,0.0,0.0,0.0,3197000,4,0.250000,0.000000e+00,0.000000e+00,0.000000e+00,7.992500e+05,3,4,1,139000.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,남성 캐주얼,관악점,남성 캐주얼,0,1,4,0.000000e+00,0.0,7.417500e+05,856750.0,0.000000e+00,0.000000e+00,0.000000e+00,0,0,2,2,0,0,0,0,0,0,남성 캐주얼,0.000000,1713500.0,2.0,8.567500e+05,1483500.0,2.0,7.417500e+05,0.0,0.0,0.000000e+00
3,3,16077620,44,365400,4935000,24,20,13,18,16,0.318182,0.590909,0.159091,0.113636,0.136364,0.0,0.0,8.932011e+05,2.444444,549620.0,4.0,1284060.0,11.0,16077620,44,0.125000,1.797523e+05,7.725571e+05,1.117080e+05,9.062700e+05,2,1,0,396630.0,14.0,1.0,1599380.0,6.0,1.0,1346550.0,6.0,1.0,45000.0,1.0,1.0,400560.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,385900.0,2.0,1.0,237000.0,2.0,1.0,기타,광주점,수산품,0,3,1,5.003333e+05,1081024.0,1.915167e+05,50308.0,1.053580e+06,6.200000e+04,1.768145e+05,3,5,6,5,5,9,11,기타,기타,수산품,수산품,0.000000,1735220.0,11.0,1.577473e+05,8582960.0,18.0,4.768311e+05,5759440.0,15.0,3.839627e+05
4,4,29050000,3,9683333,24000000,2,3,3,2,85,0.000000,0.333333,0.000000,0.666667,0.000000,0.0,0.0,1.452500e+07,1.500000,27000000.0,2.0,27000000.0,2.0,29050000,3,0.500000,2.050000e+06,0.000000e+00,1.350000e+07,0.000000e+00,3,3,0,3000000.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2050000.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,화장품,본 점,화장품,1,11,3,0.000000e+00,0.0,1.350000e+07,2050000.0,0.000000e+00,0.000000e+00,0.000000e+00,0,0,2,1,0,0,0,화장품,0,기타,0,0.000000,0.0,0.0,0.000000e+00,27000000.0,2.0,1.350000e+07,2050000.0,1.0,2.050000e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [6]:
## 연속형변수 카테고리화

### 총구매액
feature['총구매액_cat'] = 0
feature.loc[feature['총구매액']<=0.0,'총구매액_cat'] = 0
feature.loc[(feature['총구매액']>0.0) & (feature['총구매액']<=500000.0),'총구매액_cat'] = 1
feature.loc[(feature['총구매액']>500000.0) & (feature['총구매액']<=1000000.0),'총구매액_cat'] = 2
feature.loc[(feature['총구매액']>1000000.0) & (feature['총구매액']<=5000000.0),'총구매액_cat'] = 3
feature.loc[(feature['총구매액']>5000000.0) & (feature['총구매액']<=10000000.0),'총구매액_cat'] = 4
feature.loc[(feature['총구매액']>10000000.0) & (feature['총구매액']<=50000000.0),'총구매액_cat'] = 5
feature.loc[feature['총구매액']>50000000.0,'총구매액_cat'] = 6

feature = feature.drop(columns=["총구매액"])

### 구매건수
feature['구매건수_cat'] = 0
feature.loc[feature['구매건수']<=1.0,'구매건수_cat'] = 0
feature.loc[(feature['구매건수']>1.0) & (feature['구매건수']<=5.0),'구매건수_cat'] = 1
feature.loc[(feature['구매건수']>5.0) & (feature['구매건수']<=50.0),'구매건수_cat'] = 2
feature.loc[(feature['구매건수']>50.0) & (feature['구매건수']<=100.0),'구매건수_cat'] = 3
feature.loc[(feature['구매건수']>100.0) & (feature['구매건수']<=200.0),'구매건수_cat'] = 4
feature.loc[feature['구매건수']>200.0,'구매건수_cat'] = 5

feature = feature.drop(columns="구매건수")

### 평균구매액
feature['평균구매액_cat'] = 0
feature.loc[feature['평균구매액']<=0,'평균구매액_cat'] = 0
feature.loc[(feature['평균구매액']>0) & (feature['평균구매액']<=100000),'평균구매액_cat'] = 1
feature.loc[(feature['평균구매액']>100000) & (feature['평균구매액']<=500000),'평균구매액_cat'] = 2
feature.loc[(feature['평균구매액']>500000) & (feature['평균구매액']<=750000),'평균구매액_cat'] = 3
feature.loc[feature['평균구매액']>750000,'평균구매액_cat'] = 4

feature = feature.drop(columns="평균구매액")

### 최대구매액
feature['최대구매액_cat'] = 0
feature.loc[feature['최대구매액']<=0,'최대구매액_cat'] = 0
feature.loc[(feature['최대구매액']>0) & (feature['최대구매액']<=500000),'최대구매액_cat'] = 1
feature.loc[(feature['최대구매액']>500000) & (feature['최대구매액']<=1000000),'최대구매액_cat'] = 2
feature.loc[(feature['최대구매액']>1000000) & (feature['최대구매액']<=5000000),'최대구매액_cat'] = 3
feature.loc[(feature['최대구매액']>5000000) & (feature['최대구매액']<=10000000),'최대구매액_cat'] = 4
feature.loc[(feature['최대구매액']>10000000) & (feature['최대구매액']<=50000000),'최대구매액_cat'] = 5
feature.loc[(feature['최대구매액']>50000000) & (feature['최대구매액']<=100000000),'최대구매액_cat'] = 6
feature.loc[feature['최대구매액']>100000000,'최대구매액_cat'] = 7

feature = feature.drop(columns="최대구매액")

### 구매상품종류1
feature['구매상품종류1_cat'] = 0
feature.loc[feature['구매상품종류1']<=1.0,'구매상품종류1_cat'] = 0
feature.loc[(feature['구매상품종류1']>1.0) & (feature['구매상품종류1']<=5.0),'구매상품종류1_cat'] = 1
feature.loc[(feature['구매상품종류1']>5.0) & (feature['구매상품종류1']<=25.0),'구매상품종류1_cat'] = 2
feature.loc[(feature['구매상품종류1']>25.0) & (feature['구매상품종류1']<=50.0),'구매상품종류1_cat'] = 3
feature.loc[(feature['구매상품종류1']>50.0) & (feature['구매상품종류1']<=100.0),'구매상품종류1_cat'] = 4
feature.loc[feature['구매상품종류1']>100.0,'구매상품종류1_cat'] = 5

feature = feature.drop(columns="구매상품종류1")

### 구매상품종류2
feature['구매상품종류2_cat'] = 0
feature.loc[feature['구매상품종류2']<=1.0,'구매상품종류2_cat'] = 0
feature.loc[(feature['구매상품종류2']>1.0) & (feature['구매상품종류2']<=5.0),'구매상품종류2_cat'] = 1
feature.loc[(feature['구매상품종류2']>5.0) & (feature['구매상품종류2']<=10.0),'구매상품종류2_cat'] = 2
feature.loc[(feature['구매상품종류2']>10.0) & (feature['구매상품종류2']<=25.0),'구매상품종류2_cat'] = 3
feature.loc[(feature['구매상품종류2']>25.0) & (feature['구매상품종류2']<=50.0),'구매상품종류2_cat'] = 4
feature.loc[(feature['구매상품종류2']>50.0) & (feature['구매상품종류2']<=75.0),'구매상품종류2_cat'] = 5
feature.loc[feature['구매상품종류2']>75.0,'구매상품종류2_cat'] = 6

feature = feature.drop(columns="구매상품종류2")

### 구매상품종류3
feature['구매상품종류3_cat'] = 0
feature.loc[feature['구매상품종류3']<=1.0,'구매상품종류3_cat'] = 0
feature.loc[(feature['구매상품종류3']>1.0) & (feature['구매상품종류3']<=5.0),'구매상품종류3_cat'] = 1
feature.loc[(feature['구매상품종류3']>5.0) & (feature['구매상품종류3']<=10.0),'구매상품종류3_cat'] = 2
feature.loc[(feature['구매상품종류3']>10.0) & (feature['구매상품종류3']<=20.0),'구매상품종류3_cat'] = 3
feature.loc[(feature['구매상품종류3']>20.0) & (feature['구매상품종류3']<=30.0),'구매상품종류3_cat'] = 4
feature.loc[feature['구매상품종류3']>30.0,'구매상품종류3_cat'] = 5

feature = feature.drop(columns="구매상품종류3")

### 내점일수
feature['내점일수_cat'] = 0
feature.loc[feature['내점일수']<=20,'내점일수_cat'] = 0
feature.loc[(feature['내점일수']>20) & (feature['내점일수']<=40),'내점일수_cat'] = 1
feature.loc[(feature['내점일수']>40) & (feature['내점일수']<=70),'내점일수_cat'] = 2
feature.loc[(feature['내점일수']>70) & (feature['내점일수']<=100),'내점일수_cat'] = 3
feature.loc[feature['내점일수']>100,'내점일수_cat'] = 4

feature = feature.drop(columns="내점일수")

### 구매주기
feature['구매주기_cat'] = 0
feature.loc[feature['구매주기']<=20,'구매주기_cat'] = 0
feature.loc[(feature['구매주기']>20) & (feature['구매주기']<=40),'구매주기_cat'] = 1
feature.loc[(feature['구매주기']>40) & (feature['구매주기']<=70),'구매주기_cat'] = 2
feature.loc[(feature['구매주기']>70) & (feature['구매주기']<=100),'구매주기_cat'] = 3
feature.loc[feature['구매주기']>100,'구매주기_cat'] = 4

feature = feature.drop(columns="구매주기")

### 주말방문비율
feature['주말방문비율_cat'] = 0
feature.loc[feature['주말방문비율']<=0.0,'주말방문비율_cat'] = 0
feature.loc[(feature['주말방문비율']>0.0) & (feature['주말방문비율']<=0.2),'주말방문비율_cat'] = 1
feature.loc[(feature['주말방문비율']>0.2) & (feature['주말방문비율']<=0.4),'주말방문비율_cat'] = 2
feature.loc[(feature['주말방문비율']>0.4) & (feature['주말방문비율']<=0.6),'주말방문비율_cat'] = 3
feature.loc[(feature['주말방문비율']>0.6) & (feature['주말방문비율']<=0.8),'주말방문비율_cat'] = 4
feature.loc[feature['주말방문비율']>0.8,'주말방문비율_cat'] = 5

feature = feature.drop(columns="주말방문비율")

### 봄-구매비율
feature['봄-구매비율_cat'] = 0
feature.loc[feature['봄-구매비율']<=0.0,'봄-구매비율_cat'] = 0
feature.loc[(feature['봄-구매비율']>0.0) & (feature['봄-구매비율']<=0.2),'봄-구매비율_cat'] = 1
feature.loc[(feature['봄-구매비율']>0.2) & (feature['봄-구매비율']<=0.4),'봄-구매비율_cat'] = 2
feature.loc[(feature['봄-구매비율']>0.4) & (feature['봄-구매비율']<=0.6),'봄-구매비율_cat'] = 3
feature.loc[(feature['봄-구매비율']>0.6) & (feature['봄-구매비율']<=0.8),'봄-구매비율_cat'] = 4
feature.loc[feature['봄-구매비율']>0.8,'봄-구매비율_cat'] = 5

feature = feature.drop(columns='봄-구매비율')

### 여름-구매비율
feature['여름-구매비율_cat'] = 0
feature.loc[feature['여름-구매비율']<=0.0,'여름-구매비율_cat'] = 0
feature.loc[(feature['여름-구매비율']>0.0) & (feature['여름-구매비율']<=0.2),'여름-구매비율_cat'] = 1
feature.loc[(feature['여름-구매비율']>0.2) & (feature['여름-구매비율']<=0.4),'여름-구매비율_cat'] = 2
feature.loc[(feature['여름-구매비율']>0.4) & (feature['여름-구매비율']<=0.6),'여름-구매비율_cat'] = 3
feature.loc[(feature['여름-구매비율']>0.6) & (feature['여름-구매비율']<=0.8),'여름-구매비율_cat'] = 4
feature.loc[feature['여름-구매비율']>0.8,'여름-구매비율_cat'] = 5

feature = feature.drop(columns="여름-구매비율")

### 가을-구매비율
feature['가을-구매비율_cat'] = 0
feature.loc[feature['가을-구매비율']<=0.0,'가을-구매비율_cat'] = 0
feature.loc[(feature['가을-구매비율']>0.0) & (feature['가을-구매비율']<=0.2),'가을-구매비율_cat'] = 1
feature.loc[(feature['가을-구매비율']>0.2) & (feature['가을-구매비율']<=0.4),'가을-구매비율_cat'] = 2
feature.loc[(feature['가을-구매비율']>0.4) & (feature['가을-구매비율']<=0.6),'가을-구매비율_cat'] = 3
feature.loc[(feature['가을-구매비율']>0.6) & (feature['가을-구매비율']<=0.8),'가을-구매비율_cat'] = 4
feature.loc[feature['가을-구매비율']>0.8,'가을-구매비율_cat'] = 5

feature = feature.drop(columns="가을-구매비율")

### 겨울-구매비율
feature['겨울-구매비율_cat'] = 0
feature.loc[feature['겨울-구매비율']<=0.0,'겨울-구매비율_cat'] = 0
feature.loc[(feature['겨울-구매비율']>0.0) & (feature['겨울-구매비율']<=0.2),'겨울-구매비율_cat'] = 1
feature.loc[(feature['겨울-구매비율']>0.2) & (feature['겨울-구매비율']<=0.4),'겨울-구매비율_cat'] = 2
feature.loc[(feature['겨울-구매비율']>0.4) & (feature['겨울-구매비율']<=0.6),'겨울-구매비율_cat'] = 3
feature.loc[(feature['겨울-구매비율']>0.6) & (feature['겨울-구매비율']<=0.8),'겨울-구매비율_cat'] = 4
feature.loc[feature['겨울-구매비율']>0.8,'겨울-구매비율_cat'] = 5

feature = feature.drop(columns="겨울-구매비율")

### 환불금액
feature['환불금액_cat'] = 0
feature.loc[feature['환불금액']<=0.0,'환불금액_cat'] = 0
feature.loc[(feature['환불금액']>0.0) & (feature['환불금액']<=500000.0),'환불금액_cat'] = 1
feature.loc[(feature['환불금액']>500000.0) & (feature['환불금액']<=1000000.0),'환불금액_cat'] = 2
feature.loc[(feature['환불금액']>1000000.0) & (feature['환불금액']<=5000000.0),'환불금액_cat'] = 3
feature.loc[(feature['환불금액']>5000000.0) & (feature['환불금액']<=10000000.0),'환불금액_cat'] = 4
feature.loc[(feature['환불금액']>10000000.0) & (feature['환불금액']<=50000000.0),'환불금액_cat'] = 5
feature.loc[(feature['환불금액']>50000000.0) & (feature['환불금액']<=100000000.0),'환불금액_cat'] = 6
feature.loc[feature['환불금액']>100000000.0,'환불금액_cat'] = 7

feature = feature.drop(columns=["환불금액"])

### 환불건수
feature['환불건수_cat'] = 0
feature.loc[feature['환불건수']<=1,'환불건수_cat'] = 0
feature.loc[(feature['환불건수']>1) & (feature['환불건수']<=3),'환불건수_cat'] = 1
feature.loc[(feature['환불건수']>3) & (feature['환불건수']<=5),'환불건수_cat'] = 2
feature.loc[(feature['환불건수']>5) & (feature['환불건수']<=10),'환불건수_cat'] = 3
feature.loc[(feature['환불건수']>10) & (feature['환불건수']<=30),'환불건수_cat'] = 4
feature.loc[feature['환불건수']>30,'환불건수_cat'] = 5

feature = feature.drop(columns="환불건수")


### 내점당구매액
feature['내점당구매액_cat'] = 0
feature.loc[feature['내점당구매액']<=0.0,'내점당구매액_cat'] = 0
feature.loc[(feature['내점당구매액']>0.0) & (feature['내점당구매액']<=100000.0),'내점당구매액_cat'] = 1
feature.loc[(feature['내점당구매액']>100000.0) & (feature['내점당구매액']<=500000.0),'내점당구매액_cat'] = 2
feature.loc[(feature['내점당구매액']>500000.0) & (feature['내점당구매액']<=1000000.0),'내점당구매액_cat'] = 3
feature.loc[(feature['내점당구매액']>1000000.0) & (feature['내점당구매액']<=5000000.0),'내점당구매액_cat'] = 4
feature.loc[(feature['내점당구매액']>5000000.0) & (feature['내점당구매액']<=10000000.0),'내점당구매액_cat'] = 5
feature.loc[feature['내점당구매액']>10000000.0,'내점당구매액_cat'] = 6

feature = feature.drop(columns=["내점당구매액"])

### 내점당구매건수
feature['내점당구매건수_cat'] = 0
feature.loc[feature['내점당구매건수']<=1,'내점당구매건수_cat'] = 0
feature.loc[(feature['내점당구매건수']>1) & (feature['내점당구매건수']<=2),'내점당구매건수_cat'] = 1
feature.loc[(feature['내점당구매건수']>2) & (feature['내점당구매건수']<=4),'내점당구매건수_cat'] = 2
feature.loc[(feature['내점당구매건수']>4) & (feature['내점당구매건수']<=6),'내점당구매건수_cat'] = 3
feature.loc[(feature['내점당구매건수']>6) & (feature['내점당구매건수']<=10),'내점당구매건수_cat'] = 4
feature.loc[feature['내점당구매건수']>10,'내점당구매건수_cat'] = 5

feature = feature.drop(columns="내점당구매건수")


### 최근3개월_구매금액
feature['최근3개월_구매금액_cat'] = 0
feature.loc[feature['최근3개월_구매금액']<=0.0,'최근3개월_구매금액_cat'] = 0
feature.loc[(feature['최근3개월_구매금액']>0.0) & (feature['최근3개월_구매금액']<=500000.0),'최근3개월_구매금액_cat'] = 1
feature.loc[(feature['최근3개월_구매금액']>500000.0) & (feature['최근3개월_구매금액']<=1000000.0),'최근3개월_구매금액_cat'] = 2
feature.loc[(feature['최근3개월_구매금액']>1000000.0) & (feature['최근3개월_구매금액']<=10000000.0),'최근3개월_구매금액_cat'] = 3
feature.loc[(feature['최근3개월_구매금액']>10000000.0) & (feature['최근3개월_구매금액']<=50000000.0),'최근3개월_구매금액_cat'] = 4
feature.loc[(feature['최근3개월_구매금액']>50000000.0) & (feature['최근3개월_구매금액']<=100000000.0),'최근3개월_구매금액_cat'] = 5
feature.loc[feature['최근3개월_구매금액']>100000000.0,'최근3개월_구매금액_cat'] = 6

feature = feature.drop(columns=["최근3개월_구매금액"])

### 최근6개월_구매금액
feature['최근6개월_구매금액_cat'] = 0
feature.loc[feature['최근6개월_구매금액']<=0.0,'최근6개월_구매금액_cat'] = 0
feature.loc[(feature['최근6개월_구매금액']>0.0) & (feature['최근6개월_구매금액']<=500000.0),'최근6개월_구매금액_cat'] = 1
feature.loc[(feature['최근6개월_구매금액']>500000.0) & (feature['최근6개월_구매금액']<=1000000.0),'최근6개월_구매금액_cat'] = 2
feature.loc[(feature['최근6개월_구매금액']>1000000.0) & (feature['최근6개월_구매금액']<=10000000.0),'최근6개월_구매금액_cat'] = 3
feature.loc[(feature['최근6개월_구매금액']>10000000.0) & (feature['최근6개월_구매금액']<=50000000.0),'최근6개월_구매금액_cat'] = 4
feature.loc[(feature['최근6개월_구매금액']>50000000.0) & (feature['최근6개월_구매금액']<=100000000.0),'최근6개월_구매금액_cat'] = 5
feature.loc[feature['최근6개월_구매금액']>100000000.0,'최근6개월_구매금액_cat'] = 6

feature = feature.drop(columns=["최근6개월_구매금액"])

### 최근12개월_구매금액
feature['최근12개월_구매금액_cat'] = 0
feature.loc[feature['최근12개월_구매금액']<=0.0,'최근12개월_구매금액_cat'] = 0
feature.loc[(feature['최근12개월_구매금액']>0.0) & (feature['최근12개월_구매금액']<=500000.0),'최근12개월_구매금액_cat'] = 1
feature.loc[(feature['최근12개월_구매금액']>500000.0) & (feature['최근12개월_구매금액']<=1000000.0),'최근12개월_구매금액_cat'] = 2
feature.loc[(feature['최근12개월_구매금액']>1000000.0) & (feature['최근12개월_구매금액']<=10000000.0),'최근12개월_구매금액_cat'] = 3
feature.loc[(feature['최근12개월_구매금액']>10000000.0) & (feature['최근12개월_구매금액']<=50000000.0),'최근12개월_구매금액_cat'] = 4
feature.loc[(feature['최근12개월_구매금액']>50000000.0) & (feature['최근12개월_구매금액']<=100000000.0),'최근12개월_구매금액_cat'] = 5
feature.loc[feature['최근12개월_구매금액']>100000000.0,'최근12개월_구매금액_cat'] = 6

feature = feature.drop(columns=["최근12개월_구매금액"])

### 최근3개월_구매건수
feature['최근3개월_구매건수_cat'] = 0
feature.loc[feature['최근3개월_구매건수']<=1.0,'최근3개월_구매건수_cat'] = 0
feature.loc[(feature['최근3개월_구매건수']>1.0) & (feature['최근3개월_구매건수']<=10.0),'최근3개월_구매건수_cat'] = 1
feature.loc[(feature['최근3개월_구매건수']>10.0) & (feature['최근3개월_구매건수']<=25.0),'최근3개월_구매건수_cat'] = 2
feature.loc[(feature['최근3개월_구매건수']>25.0) & (feature['최근3개월_구매건수']<=50.0),'최근3개월_구매건수_cat'] = 3
feature.loc[(feature['최근3개월_구매건수']>50.0) & (feature['최근3개월_구매건수']<=80.0),'최근3개월_구매건수_cat'] = 4
feature.loc[(feature['최근3개월_구매건수']>800.0) & (feature['최근3개월_구매건수']<=100.0),'최근3개월_구매건수_cat'] = 5
feature.loc[feature['최근3개월_구매건수']>100.0,'최근3개월_구매건수_cat'] = 6

feature = feature.drop(columns="최근3개월_구매건수")

### 최근6개월_구매건수
feature['최근6개월_구매건수_cat'] = 0
feature.loc[feature['최근6개월_구매건수']<=1.0,'최근6개월_구매건수_cat'] = 0
feature.loc[(feature['최근6개월_구매건수']>1.0) & (feature['최근6개월_구매건수']<=10.0),'최근6개월_구매건수_cat'] = 1
feature.loc[(feature['최근6개월_구매건수']>10.0) & (feature['최근6개월_구매건수']<=25.0),'최근6개월_구매건수_cat'] = 2
feature.loc[(feature['최근6개월_구매건수']>25.0) & (feature['최근6개월_구매건수']<=50.0),'최근6개월_구매건수_cat'] = 3
feature.loc[(feature['최근6개월_구매건수']>50.0) & (feature['최근6개월_구매건수']<=80.0),'최근6개월_구매건수_cat'] = 4
feature.loc[(feature['최근6개월_구매건수']>800.0) & (feature['최근6개월_구매건수']<=100.0),'최근6개월_구매건수_cat'] = 5
feature.loc[feature['최근6개월_구매건수']>100.0,'최근6개월_구매건수_cat'] = 6

feature = feature.drop(columns="최근6개월_구매건수")

### 최근12개월_구매건수
feature['최근12개월_구매건수_cat'] = 0
feature.loc[feature['최근12개월_구매건수']<=1.0,'최근12개월_구매건수_cat'] = 0
feature.loc[(feature['최근12개월_구매건수']>1.0) & (feature['최근12개월_구매건수']<=10.0),'최근12개월_구매건수_cat'] = 1
feature.loc[(feature['최근12개월_구매건수']>10.0) & (feature['최근12개월_구매건수']<=25.0),'최근12개월_구매건수_cat'] = 2
feature.loc[(feature['최근12개월_구매건수']>25.0) & (feature['최근12개월_구매건수']<=50.0),'최근12개월_구매건수_cat'] = 3
feature.loc[(feature['최근12개월_구매건수']>50.0) & (feature['최근12개월_구매건수']<=80.0),'최근12개월_구매건수_cat'] = 4
feature.loc[(feature['최근12개월_구매건수']>800.0) & (feature['최근12개월_구매건수']<=100.0),'최근12개월_구매건수_cat'] = 5
feature.loc[feature['최근12개월_구매건수']>100.0,'최근12개월_구매건수_cat'] = 6

feature = feature.drop(columns="최근12개월_구매건수")

### 구매추세
feature['구매추세_cat'] = 0
feature.loc[feature['구매추세']<=0.0,'구매추세_cat'] = 0
feature.loc[(feature['구매추세']>0.0) & (feature['구매추세']<=0.2),'구매추세_cat'] = 1
feature.loc[(feature['구매추세']>0.2) & (feature['구매추세']<=0.4),'구매추세_cat'] = 2
feature.loc[(feature['구매추세']>0.4) & (feature['구매추세']<=0.6),'구매추세_cat'] = 3
feature.loc[(feature['구매추세']>0.6) & (feature['구매추세']<=0.8),'구매추세_cat'] = 4
feature.loc[feature['구매추세']>0.8,'구매추세_cat'] = 5

feature = feature.drop(columns="구매추세")

### 봄_평균구매금액
feature['봄_평균구매금액_cat'] = 0
feature.loc[feature['봄_평균구매금액']<=0.0,'봄_평균구매금액_cat'] = 0
feature.loc[(feature['봄_평균구매금액']>0.0) & (feature['봄_평균구매금액']<=500000.0),'봄_평균구매금액_cat'] = 1
feature.loc[(feature['봄_평균구매금액']>500000.0) & (feature['봄_평균구매금액']<=1000000.0),'봄_평균구매금액_cat'] = 2
feature.loc[(feature['봄_평균구매금액']>1000000.0) & (feature['봄_평균구매금액']<=10000000.0),'봄_평균구매금액_cat'] = 3
feature.loc[(feature['봄_평균구매금액']>10000000.0) & (feature['봄_평균구매금액']<=50000000.0),'봄_평균구매금액_cat'] = 4
feature.loc[(feature['봄_평균구매금액']>50000000.0) & (feature['봄_평균구매금액']<=100000000.0),'봄_평균구매금액_cat'] = 5
feature.loc[feature['봄_평균구매금액']>100000000.0,'봄_평균구매금액_cat'] = 6

feature = feature.drop(columns=["봄_평균구매금액"])

### 여름_평균구매금액
feature['여름_평균구매금액_cat'] = 0
feature.loc[feature['여름_평균구매금액']<=0.0,'여름_평균구매금액_cat'] = 0
feature.loc[(feature['여름_평균구매금액']>0.0) & (feature['여름_평균구매금액']<=500000.0),'여름_평균구매금액_cat'] = 1
feature.loc[(feature['여름_평균구매금액']>500000.0) & (feature['여름_평균구매금액']<=1000000.0),'여름_평균구매금액_cat'] = 2
feature.loc[(feature['여름_평균구매금액']>1000000.0) & (feature['여름_평균구매금액']<=10000000.0),'여름_평균구매금액_cat'] = 3
feature.loc[(feature['여름_평균구매금액']>10000000.0) & (feature['여름_평균구매금액']<=50000000.0),'여름_평균구매금액_cat'] = 4
feature.loc[(feature['여름_평균구매금액']>50000000.0) & (feature['여름_평균구매금액']<=100000000.0),'여름_평균구매금액_cat'] = 5
feature.loc[feature['여름_평균구매금액']>100000000.0,'여름_평균구매금액_cat'] = 6

feature = feature.drop(columns=["여름_평균구매금액"])

### 가을_평균구매금액
feature['가을_평균구매금액_cat'] = 0
feature.loc[feature['가을_평균구매금액']<=0.0,'가을_평균구매금액_cat'] = 0
feature.loc[(feature['가을_평균구매금액']>0.0) & (feature['가을_평균구매금액']<=500000.0),'가을_평균구매금액_cat'] = 1
feature.loc[(feature['가을_평균구매금액']>500000.0) & (feature['가을_평균구매금액']<=1000000.0),'가을_평균구매금액_cat'] = 2
feature.loc[(feature['가을_평균구매금액']>1000000.0) & (feature['가을_평균구매금액']<=10000000.0),'가을_평균구매금액_cat'] = 3
feature.loc[(feature['가을_평균구매금액']>10000000.0) & (feature['가을_평균구매금액']<=50000000.0),'가을_평균구매금액_cat'] = 4
feature.loc[(feature['가을_평균구매금액']>50000000.0) & (feature['가을_평균구매금액']<=100000000.0),'가을_평균구매금액_cat'] = 5
feature.loc[feature['가을_평균구매금액']>100000000.0,'가을_평균구매금액_cat'] = 6

feature = feature.drop(columns=["가을_평균구매금액"])

### 겨울_평균구매금액
feature['겨울_평균구매금액_cat'] = 0
feature.loc[feature['겨울_평균구매금액']<=0.0,'겨울_평균구매금액_cat'] = 0
feature.loc[(feature['겨울_평균구매금액']>0.0) & (feature['겨울_평균구매금액']<=500000.0),'겨울_평균구매금액_cat'] = 1
feature.loc[(feature['겨울_평균구매금액']>500000.0) & (feature['겨울_평균구매금액']<=1000000.0),'겨울_평균구매금액_cat'] = 2
feature.loc[(feature['겨울_평균구매금액']>1000000.0) & (feature['겨울_평균구매금액']<=10000000.0),'겨울_평균구매금액_cat'] = 3
feature.loc[(feature['겨울_평균구매금액']>10000000.0) & (feature['겨울_평균구매금액']<=50000000.0),'겨울_평균구매금액_cat'] = 4
feature.loc[(feature['겨울_평균구매금액']>50000000.0) & (feature['겨울_평균구매금액']<=100000000.0),'겨울_평균구매금액_cat'] = 5
feature.loc[feature['겨울_평균구매금액']>100000000.0,'겨울_평균구매금액_cat'] = 6

feature = feature.drop(columns=["겨울_평균구매금액"])

### 기타_구매금액
feature['기타_구매금액_cat'] = 0
feature.loc[feature['기타_구매금액']<=0.0,'기타_구매금액_cat'] = 0
feature.loc[(feature['기타_구매금액']>0.0) & (feature['기타_구매금액']<=10000.0),'기타_구매금액_cat'] = 1
feature.loc[(feature['기타_구매금액']>10000.0) & (feature['기타_구매금액']<=100000.0),'기타_구매금액_cat'] = 2
feature.loc[(feature['기타_구매금액']>100000.0) & (feature['기타_구매금액']<=500000.0),'기타_구매금액_cat'] = 3
feature.loc[(feature['기타_구매금액']>500000.0) & (feature['기타_구매금액']<=1000000.0),'기타_구매금액_cat'] = 4
feature.loc[(feature['기타_구매금액']>1000000.0) & (feature['기타_구매금액']<=5000000.0),'기타_구매금액_cat'] = 5
feature.loc[(feature['기타_구매금액']>5000000.0) & (feature['기타_구매금액']<=10000000.0),'기타_구매금액_cat'] = 6
feature.loc[feature['기타_구매금액']>10000000.0,'기타_구매금액_cat'] = 7

feature = feature.drop(columns=["기타_구매금액"])

### 농산물_구매금액
feature['농산물_구매금액_cat'] = 0
feature.loc[feature['농산물_구매금액']<=0.0,'농산물_구매금액_cat'] = 0
feature.loc[(feature['농산물_구매금액']>0.0) & (feature['농산물_구매금액']<=10000.0),'농산물_구매금액_cat'] = 1
feature.loc[(feature['농산물_구매금액']>10000.0) & (feature['농산물_구매금액']<=100000.0),'농산물_구매금액_cat'] = 2
feature.loc[(feature['농산물_구매금액']>100000.0) & (feature['농산물_구매금액']<=500000.0),'농산물_구매금액_cat'] = 3
feature.loc[(feature['농산물_구매금액']>500000.0) & (feature['농산물_구매금액']<=1000000.0),'농산물_구매금액_cat'] = 4
feature.loc[(feature['농산물_구매금액']>1000000.0) & (feature['농산물_구매금액']<=5000000.0),'농산물_구매금액_cat'] = 5
feature.loc[(feature['농산물_구매금액']>5000000.0) & (feature['농산물_구매금액']<=10000000.0),'농산물_구매금액_cat'] = 6
feature.loc[feature['농산물_구매금액']>10000000.0,'농산물_구매금액_cat'] = 7

feature = feature.drop(columns=["농산물_구매금액"])

### 가공식품_구매금액
feature['가공식품_구매금액_cat'] = 0
feature.loc[feature['가공식품_구매금액']<=0.0,'가공식품_구매금액_cat'] = 0
feature.loc[(feature['가공식품_구매금액']>0.0) & (feature['가공식품_구매금액']<=10000.0),'가공식품_구매금액_cat'] = 1
feature.loc[(feature['가공식품_구매금액']>10000.0) & (feature['가공식품_구매금액']<=100000.0),'가공식품_구매금액_cat'] = 2
feature.loc[(feature['가공식품_구매금액']>100000.0) & (feature['가공식품_구매금액']<=500000.0),'가공식품_구매금액_cat'] = 3
feature.loc[(feature['가공식품_구매금액']>500000.0) & (feature['가공식품_구매금액']<=1000000.0),'가공식품_구매금액_cat'] = 4
feature.loc[(feature['가공식품_구매금액']>1000000.0) & (feature['가공식품_구매금액']<=5000000.0),'가공식품_구매금액_cat'] = 5
feature.loc[feature['가공식품_구매금액']>5000000.0,'가공식품_구매금액_cat'] = 6

feature = feature.drop(columns=["가공식품_구매금액"])

### 축산가공_구매금액
feature['축산가공_구매금액_cat'] = 0
feature.loc[feature['축산가공_구매금액']<=0.0,'축산가공_구매금액_cat'] = 0
feature.loc[(feature['축산가공_구매금액']>0.0) & (feature['축산가공_구매금액']<=10000.0),'축산가공_구매금액_cat'] = 1
feature.loc[(feature['축산가공_구매금액']>10000.0) & (feature['축산가공_구매금액']<=100000.0),'축산가공_구매금액_cat'] = 2
feature.loc[(feature['축산가공_구매금액']>100000.0) & (feature['축산가공_구매금액']<=500000.0),'축산가공_구매금액_cat'] = 3
feature.loc[(feature['축산가공_구매금액']>500000.0) & (feature['축산가공_구매금액']<=1000000.0),'축산가공_구매금액_cat'] = 4
feature.loc[(feature['축산가공_구매금액']>1000000.0) & (feature['축산가공_구매금액']<=5000000.0),'축산가공_구매금액_cat'] = 5
feature.loc[feature['축산가공_구매금액']>5000000.0,'축산가공_구매금액_cat'] = 6

feature = feature.drop(columns=["축산가공_구매금액"])

### 수산품_구매금액
feature['수산품_구매금액_cat'] = 0
feature.loc[feature['수산품_구매금액']<=0.0,'수산품_구매금액_cat'] = 0
feature.loc[(feature['수산품_구매금액']>0.0) & (feature['수산품_구매금액']<=10000.0),'수산품_구매금액_cat'] = 1
feature.loc[(feature['수산품_구매금액']>10000.0) & (feature['수산품_구매금액']<=100000.0),'수산품_구매금액_cat'] = 2
feature.loc[(feature['수산품_구매금액']>100000.0) & (feature['수산품_구매금액']<=500000.0),'수산품_구매금액_cat'] = 3
feature.loc[(feature['수산품_구매금액']>500000.0) & (feature['수산품_구매금액']<=1000000.0),'수산품_구매금액_cat'] = 4
feature.loc[(feature['수산품_구매금액']>1000000.0) & (feature['수산품_구매금액']<=5000000.0),'수산품_구매금액_cat'] = 5
feature.loc[feature['수산품_구매금액']>5000000.0,'수산품_구매금액_cat'] = 6

feature = feature.drop(columns=["수산품_구매금액"])

### 기타_구매건수
feature['기타_구매건수_cat'] = 0
feature.loc[feature['기타_구매건수']<=1.0,'기타_구매건수_cat'] = 0
feature.loc[(feature['기타_구매건수']>1.0) & (feature['기타_구매건수']<=20.0),'기타_구매건수_cat'] = 1
feature.loc[(feature['기타_구매건수']>20.0) & (feature['기타_구매건수']<=40.0),'기타_구매건수_cat'] = 2
feature.loc[(feature['기타_구매건수']>40.0) & (feature['기타_구매건수']<=60.0),'기타_구매건수_cat'] = 3
feature.loc[(feature['기타_구매건수']>60.0) & (feature['기타_구매건수']<=80.0),'기타_구매건수_cat'] = 4
feature.loc[(feature['기타_구매건수']>80.0) & (feature['기타_구매건수']<=100.0),'기타_구매건수_cat'] = 5
feature.loc[feature['기타_구매건수']>100.0,'기타_구매건수_cat'] = 6

feature = feature.drop(columns="기타_구매건수")

### 농산물_구매건수
feature['농산물_구매건수_cat'] = 0
feature.loc[feature['농산물_구매건수']<=1.0,'농산물_구매건수_cat'] = 0
feature.loc[(feature['농산물_구매건수']>1.0) & (feature['농산물_구매건수']<=20.0),'농산물_구매건수_cat'] = 1
feature.loc[(feature['농산물_구매건수']>20.0) & (feature['농산물_구매건수']<=40.0),'농산물_구매건수_cat'] = 2
feature.loc[(feature['농산물_구매건수']>40.0) & (feature['농산물_구매건수']<=60.0),'농산물_구매건수_cat'] = 3
feature.loc[(feature['농산물_구매건수']>60.0) & (feature['농산물_구매건수']<=80.0),'농산물_구매건수_cat'] = 4
feature.loc[(feature['농산물_구매건수']>80.0) & (feature['농산물_구매건수']<=100.0),'농산물_구매건수_cat'] = 5
feature.loc[feature['농산물_구매건수']>100.0,'농산물_구매건수_cat'] = 6

feature = feature.drop(columns="농산물_구매건수")

### 가공식품_구매건수
feature['가공식품_구매건수_cat'] = 0
feature.loc[feature['가공식품_구매건수']<=1.0,'가공식품_구매건수_cat'] = 0
feature.loc[(feature['가공식품_구매건수']>1.0) & (feature['가공식품_구매건수']<=20.0),'가공식품_구매건수_cat'] = 1
feature.loc[(feature['가공식품_구매건수']>20.0) & (feature['가공식품_구매건수']<=40.0),'가공식품_구매건수_cat'] = 2
feature.loc[(feature['가공식품_구매건수']>40.0) & (feature['가공식품_구매건수']<=60.0),'가공식품_구매건수_cat'] = 3
feature.loc[(feature['가공식품_구매건수']>60.0) & (feature['가공식품_구매건수']<=80.0),'가공식품_구매건수_cat'] = 4
feature.loc[(feature['가공식품_구매건수']>80.0) & (feature['가공식품_구매건수']<=100.0),'가공식품_구매건수_cat'] = 5
feature.loc[feature['가공식품_구매건수']>100.0,'가공식품_구매건수_cat'] = 6

feature = feature.drop(columns="가공식품_구매건수")

### 축산가공_구매건수
feature['축산가공_구매건수_cat'] = 0
feature.loc[feature['축산가공_구매건수']<=1.0,'축산가공_구매건수_cat'] = 0
feature.loc[(feature['축산가공_구매건수']>1.0) & (feature['축산가공_구매건수']<=20.0),'축산가공_구매건수_cat'] = 1
feature.loc[(feature['축산가공_구매건수']>20.0) & (feature['축산가공_구매건수']<=40.0),'축산가공_구매건수_cat'] = 2
feature.loc[(feature['축산가공_구매건수']>40.0) & (feature['축산가공_구매건수']<=60.0),'축산가공_구매건수_cat'] = 3
feature.loc[(feature['축산가공_구매건수']>60.0) & (feature['축산가공_구매건수']<=80.0),'축산가공_구매건수_cat'] = 4
feature.loc[(feature['축산가공_구매건수']>80.0) & (feature['축산가공_구매건수']<=100.0),'축산가공_구매건수_cat'] = 5
feature.loc[feature['축산가공_구매건수']>100.0,'축산가공_구매건수_cat'] = 6

feature = feature.drop(columns="축산가공_구매건수")

### 수산품_구매건수
feature['수산품_구매건수_cat'] = 0
feature.loc[feature['수산품_구매건수']<=1.0,'수산품_구매건수_cat'] = 0
feature.loc[(feature['수산품_구매건수']>1.0) & (feature['수산품_구매건수']<=20.0),'수산품_구매건수_cat'] = 1
feature.loc[(feature['수산품_구매건수']>20.0) & (feature['수산품_구매건수']<=40.0),'수산품_구매건수_cat'] = 2
feature.loc[(feature['수산품_구매건수']>40.0) & (feature['수산품_구매건수']<=60.0),'수산품_구매건수_cat'] = 3
feature.loc[(feature['수산품_구매건수']>60.0) & (feature['수산품_구매건수']<=80.0),'수산품_구매건수_cat'] = 4
feature.loc[(feature['수산품_구매건수']>80.0) & (feature['수산품_구매건수']<=100.0),'수산품_구매건수_cat'] = 5
feature.loc[feature['수산품_구매건수']>100.0,'수산품_구매건수_cat'] = 6

feature = feature.drop(columns="수산품_구매건수")

In [7]:
def f_1(x,i,a):
    a = 0
    if x < feature.iloc[:,-1].quantile(0.1):
        return 0
    elif x < feature.iloc[:,-1].quantile(0.3):
        return 1
    elif x < feature.iloc[:,-1].quantile(0.5):
        return 2
    elif x < feature.iloc[:,-1].quantile(0.7):
        return 3
    elif x < feature.iloc[:,-1].quantile(0.9):
        return 4
    else:
        return 5

In [8]:
a = 0
for i in [-1,-2,-3,-4,-5,-6,-7,-8,-9]:
    feature.iloc[:,i] = feature.iloc[:,i].apply(f_1,args=(i,a))

In [9]:
def f_2(x):
    if x <= 20:
        return 0
    elif x < 30:
        return 1
    elif x < 40:
        return 2
    elif x < 50:
        return 3
    elif x < 60:
        return 4
    elif x < 70:
        return 5
    elif x < 80:
        return 6
    else :
        return 7

In [10]:
feature['다른지점구매비율'] = feature['다른지점구매비율'].apply(lambda x: x*100)
feature['다른지점구매비율'] = feature['다른지점구매비율'].apply(f_2)

In [11]:
for i in [-15,-16,-17,-18,-19,-20,-21,-22,-23,-24,-25,-26,-27,-28]:
    feature.iloc[:,i] = feature.iloc[:,i].apply(f_1,args=(i,a))

In [12]:
for i in [-35,-36,-37,-38,-39,-40,-41,-42,-43,-44,-45,-46,-47,-48,-49]:
    feature.iloc[:,i] = feature.iloc[:,i].apply(f_1,args=(i,a))

In [13]:
feature.to_csv('features.csv',encoding='cp949')